##PINN:

In [1]:
import time
import math
import torch
import numpy as np
from problems import sixth_order
from pinn import PINN, make_points
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
np.random.seed(1234)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
#domain setting
x_0=-10.
x_1=10.
t_0=0.
t_1=1.
#problem setting
a1 = 1
a2 = -1
a4 = -0.3
a6 = 0.1
b1 = 6
khi = 1
a_param = 0.5
x0_param = 4
theta0 = math.pi/3
problem = sixth_order(x_0, x_1, t_0, t_1, a1, a2, a4, a6, b1, khi, a_param, x0_param, theta0)
#info about the problem
problem.show_params()

a1 = 1.000, a2 = -1.000, a3 = -2.800, a4 = -0.300, a5 = -0.600, a6 = 0.100,
 b1 = 6.000, b2 = -1.525, b3 = 0.113


In [3]:
#data for training
X_i_train, U_i_train, V_i_train, X_b_train, X_grid = make_points(problem, 400, 100, grid_resolution_x=200, grid_resolution_t=100)
#data for metrics evaluation
x_parts=200
t_parts=100
x=np.linspace(x_0,x_1,x_parts)
t=np.linspace(t_0,t_1,t_parts)
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) #test input for PINN
Q_truth=problem.q(X,T) #ground truth
layers = [2, 100, 100, 100, 2] #network topology

In [4]:
#rel_h evaluation for tuning
def show_rel_h(model):
    u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star)
    Q_pred=u_pred.reshape((t_parts, x_parts)) + 1j*v_pred.reshape((t_parts, x_parts))
    Q_abs_pred=np.abs(Q_pred)
    Q_abs_truth=np.abs(Q_truth)
    rel_h = np.linalg.norm(Q_abs_truth.flatten() - Q_abs_pred.flatten(), 2)/np.linalg.norm(Q_abs_truth.flatten(), 2)
    return rel_h

In [5]:
def objective(config):  #Wrap a PyTorch model in an objective function.
    model = PINN(problem, layers, X_i_train, U_i_train, V_i_train, X_b_train, X_grid)
    ##Setting hyperparameters
    #verbosity
    model.verbosity = 10000 #loss output frequency
    model.make_res_gif = False #makes gif with residual history
    #points generation options
    model.points_gen_method = config["points_gen_method"] #"random"/first"/"second"/"third"
    model.points_gen_freq = 10 #points generation frequency
    model.points_am = 5000 #amount of collocation points
    #optimization options
    model.adam_steps = 10000
    model.lbfgs_steps = 0
    model.nncg_steps = 0
    model.adam_step_decay = 0.997
    model.lbfgs_step_decay = 0.990
    model.nncg_step_decay = 0.990
    model.decay_freq = 100
    #loss balancing options
    model.loss_bal_method = "none" #"none"/"relobralo"
    model.bal_freq = 1 #loss rebalancing frequency
    model.lambda_i = 10/12
    model.lambda_b = 1/12
    model.lambda_f = 1/12
    model.extinction = 0.9 #extinction coefficient for ReLoBRaLo
    #causal training
    model.causal_loss = False
    model.epsilon = 0.10
    model.t_partition = 30 #number of parts in the [t_0, t_1] division
    
    while True:
        model.train()
        rel_h = show_rel_h(model)
        model.clear()
        train.report({"rel_h": rel_h})

search_space = {"points_gen_method": tune.choice(["random","first","second","third"])}

trainable_with_gpu = tune.with_resources(objective, {"cpu": 10, "gpu": 1})
tuner = tune.Tuner(objective,
    tune_config=tune.TuneConfig(metric="rel_h", mode="min", search_alg=OptunaSearch(), num_samples=6),
    run_config=train.RunConfig(stop={"training_iteration": 3}),
    param_space=search_space)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)

(objective pid=15349) Training started with third points generation method and none loss balancing
(objective pid=15349) 10000 steps of ADAM:
(objective pid=15515) Training started with second points generation method and none loss balancing
(objective pid=15515) Iter 10000, Loss: 8.674e-03, Loss_i: 2.87e-03, Loss_b: 4.37e-04, Loss_f: 7.50e-02
(objective pid=15515) 10000 steps of ADAM:
(objective pid=15515) Total iterations: 10000 + 0 + 0
(objective pid=15515) Training started with second points generation method and none loss balancing
(objective pid=15515) 10000 steps of ADAM:
(objective pid=15349) Iter 10000, Loss: 4.234e-03, Loss_i: 1.56e-03, Loss_b: 1.02e-04, Loss_f: 3.51e-02
(objective pid=15349) Total iterations: 10000 + 0 + 0
(objective pid=15349) Training started with third points generation method and none loss balancing
(objective pid=15349) 10000 steps of ADAM:
(objective pid=15515) Iter 10000, Loss: 3.538e-03, Loss_i: 1.28e-03, Loss_b: 1.84e-04, Loss_f: 2.95e-02
(objective

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

